Upload AirNow data
------------------

Processes the Airnow hourly data files located in `./AirNow` and uploads to ESDR.

Reports to stat.createlab.org as `UploadAirnowToEsdr`

[Format documentation](http://www.airnowapi.org/docs/HourlyDataFactSheet.pdf)

In [2]:
import json, os, dateutil, re, requests, subprocess, datetime, glob, stat, codecs, sys

from dateutil import rrule, tz, parser

In [3]:
# Boilerplate to load utils.ipynb
# See https://github.com/CMU-CREATE-Lab/python-utils/blob/master/utils.ipynb


def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    if(nb['nbformat'] >= 4):
        src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    else:
        src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']

    tmpname = '/tmp/%s-%s-%d.py' % (os.path.basename(filename_or_url),
                                    datetime.datetime.now().strftime('%Y%m%d%H%M%S%f'),
                                    os.getpid())
    src = '\n\n\n'.join(src)
    open(tmpname, 'w').write(src)
    code = compile(src, tmpname, 'exec')
    exec(code, globals())


exec_ipynb('./python-utils/utils.ipynb')
exec_ipynb('./python-utils/esdr-library.ipynb')
exec_ipynb('./airnow-common.ipynb')

In [4]:
STAT_SERVICE_NAME = 'UploadAirnowToEsdr'
STAT_HOSTNAME = 'airnow'
STAT_SHORTNAME = 'airnow-hourly-data-upload'

MIRROR_TIME_PERIOD_SECS = 60 * 30 # every 30 minutes

ESDR_MONITORING_SITE_LOCATION_DEVICES_JSON_FILENAME = 'esdr_monitoring_site_location_devices.json'

In [5]:
Stat.set_service(STAT_SERVICE_NAME)

In [6]:
# Accumulate data from multiple files
# Assumes accumulation in time order
accumulated = {}
accumulated_files = {}

def clear_accumulated():
    global accumulated, accumulated_files
    accumulated = {}
    accumulated_files = {}

def accumulate_airnow_file(src):
    print('Accumulating airnow file %s' % src)
    src_epoch_timestamp = os.path.getmtime(src)
    dt = datetime.datetime.strptime(os.path.basename(src), '%Y%m%d%H.dat')
    # Offset epoch_time by 1800 seconds to be in middle of hour-long sample
    epoch_time = (dt - datetime.datetime(1970, 1, 1)).total_seconds() + 1800

    nsamples = 0

    with open(src, 'r', encoding='cp437') as airnow:
        lineno = 0
        error_count = 0
        for record in airnow:
            lineno += 1
            try:
                (_, _, id, _, _, type, units, value, _) = record.split('|')
            except:
                sys.stderr.write('Problem parsing %s line %d, skipping\n' % (src, lineno))
                sys.stderr.write('Line "%s"\n' % record)
                error_count += 1
            type = re.sub(r'\W', '_', type) # Replace non-word chars with _;  e.g. PM2.5 becomes PM2_5

            if not id in accumulated:
                accumulated[id] = {}

            if not type in accumulated[id]:
                accumulated[id][type] = []

            accumulated[id][type].append([epoch_time, float(value)])
            nsamples += 1
        if error_count > 5:
            raise Exception('Too many parse errors (%d) reading %s, aborting' % (error_count, src))

    if error_count > 0:
        Stat.warning('Read %d records from %s (%d error(s))' % (nsamples, src, error_count), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    else:
        Stat.debug('Read %d records from %s (%d error(s))' % (nsamples, src, error_count), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)

    accumulated_files[src] = src_epoch_timestamp

In [7]:
sites_cached = None
def get_site_info(site_id):
    global sites_cached
    if not sites_cached:
        with open(AirnowCommon.DATA_DIRECTORY + '/monitoring_site_locations.json', 'r') as f:
            sites_cached = json.load(f)

    try:
        return sites_cached['sites'][site_id]
    except:
        return None

# print(json.dumps(get_site_info('420030008'), sort_keys=True, indent=3))  # Lawrenceville aka "BAPC 301 39TH STREET BLDG #7 AirNow"
# print(json.dumps(get_site_info('000050121'), sort_keys=True, indent=3))  # Meteorological Service of Canada"
# print(json.dumps(get_site_info('044201010'), sort_keys=True, indent=3))  # null

In [8]:
esdr = None
airnow_product = None
esdr_monitoring_site_devices = None

In [9]:
def get_airnow_product():
    global esdr, airnow_product
    if not esdr:
        esdr = Esdr('esdr-auth-airnow-uploader.json', user_agent='esdr-library.py['+STAT_SERVICE_NAME+']')
    if not airnow_product:
        # esdr.create_product('AirNow', 'AirNow', 'EPA and Sonoma Tech', 'Real-time feeds from EPA/STI AirNow')
        airnow_product = esdr.get_product_by_name('AirNow')
    return airnow_product

In [10]:
def get_esdr_monitoring_site_device(serialNumber):
    global airnow_product, esdr_monitoring_site_devices
    if not airnow_product:
        airnow_product = get_airnow_product()
    if not esdr_monitoring_site_devices:
        with open(AirnowCommon.DATA_DIRECTORY + '/' + ESDR_MONITORING_SITE_LOCATION_DEVICES_JSON_FILENAME, 'r') as f:
            esdr_monitoring_site_devices = json.load(f)

    if serialNumber in esdr_monitoring_site_devices:
        # get a copy of the device
        device = esdr_monitoring_site_devices[serialNumber].copy()

        # add the serial number and product id
        device['serialNumber'] = serialNumber
        device['productId'] = airnow_product['id']
        return device

    return None

# print(get_esdr_monitoring_site_device('no such site'))  # None
# print(get_esdr_monitoring_site_device('010972005'))     # {'id': 2264, 'name': 'BAYROAD', 'serialNumber': '010972005', 'productId': 11}

In [11]:
def upload_site(site_id):
    global esdr, airnow_product
    if not esdr:
        esdr = Esdr('esdr-auth-airnow-uploader.json', user_agent='esdr-library.py['+STAT_SERVICE_NAME+']')
    if not airnow_product:
        airnow_product = get_airnow_product()

    # try to get the device from the cache
    device = get_esdr_monitoring_site_device(site_id)

    site_info = get_site_info(site_id)

    if not device:
        if not site_info:
            Stat.warning('Cannot create device for site %s because no information can be found for it.  Skipping.' % (site_id), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
            return
        device = esdr.get_or_create_device(airnow_product, serial_number=site_id, name=site_info['site name'])

    # find the feed, but give the lat/lon for the case where the site has moved, because esdr.get_feed()
    # will match by lat/lon if there are multiple feeds for the device
    lat = float(site_info['latitude']) if site_info else None
    lon = float(site_info['longitude']) if site_info else None
    feed = esdr.get_feed(device, lat=lat, lon=lon)

    if not feed:
        if not site_info:
            Stat.warning('Cannot create feed for site %s because no information can be found for it.  Skipping.' % (site_id), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
            return
        feed = esdr.get_or_create_feed(device, lat=lat, lon=lon)

    if site_id in accumulated:
        channels = accumulated[site_id]

        for channel in channels:
            try:
                esdr.upload(feed, {
                    'channel_names': [channel],
                    'data': channels[channel]
                })
                Stat.info('%s/%s, %s: Uploaded %d samples.' % (site_id, device['name'], channel, len(channels[channel])), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
            except requests.HTTPError as e:
                Stat.warning('%s/%s, %s: Failed to upload %d samples (HTTP %d).' %
                             (site_id, device['name'], channel, len(channels[channel]), e.response.status_code), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
            except:
                Stat.warning('%s/%s, %s: Failed to upload %d samples.' %
                             (site_id, device['name'], channel, len(channels[channel])), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    else:
        Stat.warning('%s/%s: No accumulated data found. Skipping.' %
                             (site_id, device['name']), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)

#upload_site('000010401')
#upload_site('000051501')

In [12]:
def upload_check_path(src):
    return 'upload-airnow-to-esdr/uploaded-' + os.path.basename(src)

def upload_accumulated():
    i = 0
    for site_id in sorted(accumulated.keys()):
        print('Uploading site %d' % i)
        i += 1
        upload_site(site_id)
    for src in sorted(accumulated_files):
        check_path = upload_check_path(src)
        try:
            os.makedirs(os.path.dirname(check_path))
        except:
            pass
        open(check_path + '.tmp', 'w').close()
        src_epoch_time = accumulated_files[src]
        os.utime(check_path + '.tmp', (src_epoch_time, src_epoch_time))
        os.rename(check_path + '.tmp', check_path)
        Stat.debug('Uploaded %s to ESDR' % (src), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    clear_accumulated()
    return i

def process_all():
    Stat.up('Uploading hourly Airnow data to ESDR...', host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    before = time.time()
    clear_accumulated()
    for src in sorted(glob.glob('AirNow/[0-9]*.dat')):
        if len(accumulated_files) == 1000:
            upload_accumulated()
        try:
            if os.path.getmtime(src) == os.path.getmtime(upload_check_path(src)):
                continue
        except:
            pass

        accumulate_airnow_file(src)
    nsites = upload_accumulated()
    after = time.time()
    Stat.up('Done uploading %d sites to ESDR' % nsites, details='Took %.1f minutes' % ((after - before) / 60), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME, valid_for_secs=MIRROR_TIME_PERIOD_SECS*1.5)

def process_all_forever():
    while True:
        process_all()
        sleep_until_next_period(MIRROR_TIME_PERIOD_SECS)

process_all_forever()

Stat.log up UploadAirnowToEsdr airnow Beginning upload of hourly Airnow data to ESDR... None


Accumulating airnow file AirNow/2020030302.dat
Stat.log debug UploadAirnowToEsdr airnow Read 14688 records from AirNow/2020030302.dat (0 error(s)) None


Accumulating airnow file AirNow/2020030303.dat


Stat.log debug UploadAirnowToEsdr airnow Read 14866 records from AirNow/2020030303.dat (0 error(s)) None


Accumulating airnow file AirNow/2020030304.dat
Stat.log debug UploadAirnowToEsdr airnow Read 7387 records from AirNow/2020030304.dat (0 error(s)) None


...

Uploading site 42


Stat.log info UploadAirnowToEsdr airnow 000053901/Édouard, OZONE: Uploaded 39 samples. None


Uploading site 43


Stat.log info UploadAirnowToEsdr airnow 000054201/Chapais, OZONE: Uploaded 40 samples. None


Uploading site 44
